In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


df = pd.read_csv('YoutubeCommentsDataSet_Filtered.csv')

print(df.head())

                                             Comment Sentiment
0  lets not forget that apple pay in 2014 require...   neutral
1  i will forever acknowledge this channel with t...  positive
2  apple pay is so convenient secure and easy to ...  positive
3  for now i need both apple pay and the physical...   neutral
4  in the united states we have an abundance of r...  positive


In [2]:
# Lets map the 'Sentiment' label to numerical format
# 'positive': 1, 'neutral': 0, and 'negative': -1

In [3]:
sentiment_map = {"positive": 1, "neutral": 0, "negative": -1}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

In [4]:
df.head()

,Comment,Sentiment,Sentiment_Score
0,lets not forget that apple pay in 2014 require...,neutral,0
1,i will forever acknowledge this channel with t...,positive,1
2,apple pay is so convenient secure and easy to ...,positive,1
3,for now i need both apple pay and the physical...,neutral,0
4,in the united states we have an abundance of r...,positive,1


In [5]:
# Preprocess text
# (tokenization, stopwords removal, lemmatization)
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a string
    preprocessed_text = " ".join(lemmatized_tokens)

    return preprocessed_text

In [6]:
# Lets preprocess the text
# first: tokenize the text
# second: remove stop words
# third: stemming

In [7]:
df['Comment'].apply(preprocess_text)

0        let forget apple pay 2014 required brand new i...
1        forever acknowledge channel help lesson idea e...
2        apple pay convenient secure easy use used kore...
3                      need apple pay physical credit card
4        united state abundance retailer accept apple p...
                               ...                        
14309    came machine learning related talk got life le...
14310    ive worked 2 year mlcomputer vision lot thing ...
14311    come physic background usually spend lot time ...
14312    hey daniel discovered channel couple day ago i...
14313    great focus key playful approach also speed th...
Name: Comment, Length: 14314, dtype: object

In [10]:
# Separating the feature and label
X = df["Comment"]
y = df['Sentiment_Score']